## Federated Learning with a trusted aggregator

Step 1: Create Data Owners

In [2]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn,optim

#create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

#A toy dataset
data = th.tensor([[0,0], [0,1], [1,0], [1,1.]], requires_grad=True)
target = th.tensor([[0], [0], [1], [1.]], requires_grad=True)

#get pointers to training data on each worker by sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

W0801 14:58:57.522418  5424 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Vilas_2\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0801 14:58:57.598405  5424 deprecation_wrapper.py:119] From C:\Users\Vilas_2\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0801 14:59:03.845788  5424 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0801 14:59:03.847821  5424 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0801 14:59:03.849819  5424 base.py:628] Worker bob already exists. Replacing old worker which could cause         

Step 2: Create our model

In [3]:
#Initialize a toy model
model = nn.Linear(2,1)

Step 3: Send a copy of the model to Alice and Bob

In [4]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

Step 4: Train Bob's and Alice's models in parallel

In [6]:
for i in range(10):
    #Train Bob's model
    bobs_opt.zero_grad()
    bobs_pred = bobs_model(bobs_data)
    bobs_loss = ((bobs_pred - bobs_target)**2).sum()
    bobs_loss.backward()
    
    bobs_opt.step()
    bobs_loss = bobs_loss.get().data
    
    #Train Alice's model
    alices_opt.zero_grad()
    alices_pred = alices_model(alices_data)
    alices_loss = ((alices_pred - alices_target)**2).sum()
    alices_loss.backward()
    
    alices_opt.step()
    alices_loss = alices_loss.get().data

Step 5: Send both updated models to secure worker

In [7]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

Step 6: Average the model

In [8]:
with th.no_grad():
    model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) /2).get())
    model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) /2).get())

Step 7: Rinse and Repeat

In [10]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)
    
    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)
    
    for wi in range(worker_iters):
        #Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()
        
        bobs_opt.step()
        bobs_loss = bobs_loss.get().data
        
        #Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()
        
        alices_opt.step()
        alices_loss = alices_loss.get().data
        
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    
    with th.no_grad():
        
        model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) /2).get())
        model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) /2).get())
        
    print("Bob:" + str(bobs_loss) + "Alice:" + str(alices_loss))

Bob:tensor(0.0089)Alice:tensor(0.0010)
Bob:tensor(0.0075)Alice:tensor(0.0007)
Bob:tensor(0.0062)Alice:tensor(0.0004)
Bob:tensor(0.0050)Alice:tensor(0.0003)
Bob:tensor(0.0040)Alice:tensor(0.0002)
Bob:tensor(0.0032)Alice:tensor(0.0001)
Bob:tensor(0.0025)Alice:tensor(7.7928e-05)
Bob:tensor(0.0019)Alice:tensor(5.4667e-05)
Bob:tensor(0.0015)Alice:tensor(3.9136e-05)
Bob:tensor(0.0012)Alice:tensor(2.8477e-05)


In [11]:
preds = model(data)
loss = ((preds - target) ** 2).sum()

In [12]:
print(preds)
print(target)
print(loss.data)

tensor([[0.0968],
        [0.0794],
        [0.8998],
        [0.8824]], grad_fn=<AddmmBackward>)
tensor([[0.],
        [0.],
        [1.],
        [1.]], requires_grad=True)
tensor(0.0395)


## Intro to Additive Secret Sharing

In [13]:
x = 5

In [14]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

In [15]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

In [16]:
#encrypted "5"
bob_x_share = 2
alice_x_share = 3

#encrypted "7"
bob_y_share = 5
alice_y_share = 2

#encrypted 5+7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

In [17]:
#to fix a little leak from encryption
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 #a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [18]:
(bob_x_share + alice_x_share) % Q

5

## Build Methods for Encrypt, Decrypt and Add

In [19]:
x_share = (2,5,7)

In [20]:
import random

Q = 23740629843760239486723

def encrypt(x, n_share=3):
    
    shares = list()
    
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))
        
        shares.append(Q-(sum(shares) % Q) + x)
        
        return tuple(shares)
    
def decrypt(shares):
    return sum(shares) % Q

In [21]:
shares = encrypt(3)
shares

(23541065525749438853960, 199564318010800632766)

In [22]:
decrypt(shares)

3

In [24]:
def add(a,b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [25]:
x = encrypt(5)
y = encrypt(7)
z = add(x,y)
decrypt(z)

12

## Intro to Fixed Precision Encoding

In [26]:
BASE = 10
PRECISION = 4

In [27]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE ** PRECISION 

In [28]:
encode(3.5)

35000

In [30]:
decode(35000)

3.5

In [31]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

## Secret Sharing + Fixed Precision in Pysyft

In [32]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [34]:
x = th.tensor([1,2,3,4,5])

Secret sharing using pysyft

In [35]:
x = x.share(bob, alice, secure_worker)

In [36]:
bob._objects

{55098120566: tensor([1838277405673891954, 4148051163080581910, 2636776790407520613,
         3654684529539620120, 3324431514579164873])}

In [37]:
y = x + x

In [38]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:3570144196 -> bob:47686223742]
	-> (Wrapper)>[PointerTensor | me:96777537672 -> alice:77953692657]
	-> (Wrapper)>[PointerTensor | me:41137480033 -> secure_worker:39097724757]
	*crypto provider: me*

In [39]:
y.get()

tensor([ 2,  4,  6,  8, 10])

Fixed Precision using Pysyft

In [40]:
x = th.tensor([0.1, 0.2,0.3])

In [41]:
x

tensor([0.1000, 0.2000, 0.3000])

In [42]:
x = x.fix_prec()

In [43]:
x.child.child

tensor([100, 200, 300])

In [44]:
y = x+x

In [46]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

## Shared Fixed Precision

In [47]:
x = th.tensor([0.1, 0.2, 0.3])

In [48]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [49]:
y = x+x

In [50]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])